In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

在训练过程中，使用来自伯努利分布的样本，以概率p随机地将输入张量的一些元素归零， 即以概率p丢掉一些输入
```Python
m = nn.Dropout(p=0.2)
input = m(input)
```
大约就会有 20 % 的 值被归为0

在预测的时候，为了保证输出期望相同，所以需要乘 1-p, 因为预测的时候，是所有的神经元都参与的。
但实际应用中，我们希望 预测的时候尽可能块，尽可能的少计算，所以现有的框架中，对dropout的操作都是在训练的时候，就直接 除 (1-p)，这样在预测的时候就不用再做多余的计算了.
即把预测时的 **乘 (1-p)** 放到训练阶段，做了**除 (1-p)**

In [1]:
import numpy as np


class Dropout:
    def __init__(self, p, training=True):
        """p 丢弃的概率
        """
        self.p = p
        self.training = training

    def forward(self, input_):
        if self.training:
            batch_size, features = input_.shape
            # self.p 是丢弃的概率，1-self.p 是我们保留的概率
            mask = np.random.binomial(1, 1 - self.p, (batch_size, features))
            # 元素相乘，即表示 丢弃一部分输入
            output = mask * input_
            output = output / (1-self.p)
        else:
            output = input_
        return output

In [ ]:
import numpy as np


class Dropout:
    def __init__(self, p):
        """p 丢弃的概率
        """
        self.p = p

    def forward(self, input_):
        batch_size, features = input_.shape
        # self.p 是丢弃的概率，1-self.p 是我们保留的概率
        mask = np.random.binomial(1, 1 - self.p, (batch_size, features))
        # 元素相乘，即表示 丢弃一部分输入
        output = mask * input_
        return output

    def inference(self, input_):
        # 为了保证 和训练阶段的期望一致
        return input_ / (1-self.p)